In [25]:
import numpy as np
import matplotlib.pyplot as plt

h = 60 #cm
b = 60 #cm
fprimc = 240 #kg/cm2
fy = 4200 #kg/cm2
Asi = [11.59, 7.1, 7.1, 7.1, 7.1, 7.1, 11.59]
diam_var_extrem = 1.4 #cm
rec = 4 #cm
diam_est = 1 #cm

Mu_res = 51.52 #T*m
Pu_res = 472.37 #T

As_1porcient = (b*h)*0.01 #%
As_asumido = np.sum(Asi) #cm
porcent_As_real = (As_asumido/(b*h))*100 #%

dt = h - diam_est - rec - (diam_var_extrem*0.5)
db = diam_est + rec + (diam_var_extrem*0.5)
X = (dt - db)/(len(Asi)-1)

di = np.zeros(len(Asi))
for i in range (2,len(Asi),1):
    di[1] = dt
    di[i] = dt - (X*(i-1))

y = 1
c = np.zeros(81)
for i in np.linspace(5, h+20, num=h+20, endpoint=False):
    c[y] = i
    y+=1
fsi = np.zeros(len(c))


di = np.zeros((len(Asi), 1))
for i in range(1, len(Asi)):
    di[i] = di[i-1] + 0.1

fsi = np.zeros((len(Asi), len(c)))
for y in range(1, len(c)):
    for i in range(1, len(Asi)):
        fsi[i, y] = 6120 * ((c[y] - di[i, 0]) / c[y])


# fsi = np.zeros((len(Asi), len(c)))
# for y in range(1, len(c)):
#     for i in range(1, len(Asi)):
#         fsi[i, y] = 6120 * ((c[y] - di[i, 1]) / c[y])


# fsi_c = np.zeros(len(Asi))
# for y in range (1, len(c)):
#     for i in range (1,len(Asi)):
#         if fsi[i,y] > fy:
#             fsi_c[i,y] = fy
#         elif fsi[i,y] < -fy:
#             fsi_c[i,y] = -fy
#         else:
#             fsi_c[i,y] = fsi[i,y]

fsi_c = np.zeros((len(Asi), len(c)))
for y in range(1, len(c)):
    for i in range(1, len(Asi)):
        if fsi[i, y] > fy:
            fsi_c[i, y] = fy
        elif fsi[i, y] < -fy:
            fsi_c[i, y] = -fy



# force = np.zeros(len(Asi))
# for y in range (1,len(c)):
#     for i in range (1,len(Asi)):
#         force[i,y] = fsi_c[i,y] * Asi[i,1]

Asi = np.zeros((len(Asi), 1))
for i in range(1, len(Asi)):
    Asi[i] = Asi[i-1] + 0.1

force = np.zeros((len(c), len(Asi)))
for i in range(len(c)):
    for j in range(len(Asi)):
        force[i, j] = fsi_c[i, j] * Asi[j]

P_sum = np.zeros(len(c))
for i in range(len(c)):
    P_sum[i] = np.sum(force[i])

Arm_di = np.zeros(len(Asi))
for i in range(len(Asi)):
    Arm_di[i] = np.sum(force[:, i])

Moment = np.zeros(len(Asi))
for y in range (1,len(c)):
    for i in range (1,len(Asi)):
        Moment[i,y] = fsi_c[i,y]*Asi[i,1]*Arm_di[i,1]

M_sum = np.zeros(len(c))
for i in range (1,len(c)):
    M_sum[i,1] = np.sum(Moment[:,i])

ety = np.zeros(len(c))
for i in range (1,len(c)):
    ety[i,1] = 0.003*((di[1,1]-c[i,1])/c[i,1])

fi = np.zeros(len(c))
for i in range (1,len(c)):
    if ety[i,1] < 0.0021:
        fi[i,1] = 0.65
    elif ety[i,1]>0.005:
        fi[i,1] = 0.9
    else:
        fi[i,1] = 0.65 + ((0.25*(ety[i,1]-0.0021))/(0.005-0.002))


Pn = np.zeros(len(c))
Pu = np.zeros(len(c))
for i in range (1,len(c)):
    Pn[i,1] = ((0.85*fprimc*0.85*c[i,1]*b)+P_sum[i,1])/1000
    Pu[i,1] = Pn[i,1]*fi[i,1]

Mn = np.zeros(len(c))
Mu = np.zeros(len(c))
for i in range (1, len(c)):
    Mn[i,1] = ((0.85*fprimc*0.85*c[i,1]*b*((h/2)-((0.85*c[i,1])/2))) + M_sum[i,1])/100000
    Mu[i,1] = Mn[i,1]*fi[i,1]

Mn_res = Mu_res/0.9
Pn_res = Pu_res/0.9

# Import necessary libraries
import matplotlib.pyplot as plt

# Define variables
Eje_X = [0, max(Mn)]
Eje_Y = [0, max(Pn)]

# Create a figure
fig = plt.figure(1)

# Hold on to the current axes
plt.hold(True)

# Plot the nominal diagram
plt.plot(Mn[:, 1], Pn[:, 1], color=[0, 0, 0], linewidth=2)

# Plot the ultimate diagram
plt.plot(Mu[:, 1], Pu[:, 1], color=[0, 0, 1], linewidth=2)

# Plot the demand Pu and Mu
plt.plot(Mu_res, Pu_res, 'o', markeredgecolor=[0, 1, 0], markerfacecolor=[1, 0.6, 0.6])

# Plot the axes
plt.plot(Eje_X, [0, 0], color=[0, 0, 0])
plt.plot([0, 0], Eje_Y, color=[0, 0, 0])

# Add a legend
plt.legend(['Nominal Diagram', 'Ultimate Diagram', 'Demand Pu and Mu'], loc='best')

# Turn on the box
plt.box(on=True)

# Add a title
plt.title('Nominal and Ultimate Interaction Diagram')

# Add x and y labels
plt.xlabel('Mn')
plt.ylabel('Pn')

# Get the current figure handle
fig = plt.gcf()

# Set the figure units to normalized
fig.Units = 'normalized'

# Set the figure outer position
fig.OuterPosition = [0.5, 0.5, 0.5, 0.5]

# Set the loose inset of the axes to the tight inset
set(gca, 'LooseInset', get(gca, 'TightInset'))

# Set the figure to be visible
set(gcf, 'visible', 'on')

# Turn on the grid
plt.grid(True)

# Hold off
plt.hold(False)

# Show the plot
plt.show()


IndexError: index 7 is out of bounds for axis 0 with size 7